# balanced network

Let's start with a rather complete example modified from 

http://neuralensemble.org/docs/PyNN/examples/VAbenchmarks.html


## initialization

In [1]:
from pyNN.utility import Timer, ProgressBar, init_logging, normalized_filename
from pyNN.random import NumpyRNG, RandomDistribution

import pyNN.nest as sim

init_logging(None, debug=False)

timer = Timer()

# === Define parameters ========================================================

threads  = 1
rngseed  = 98765
parallel_safe = True

n        = 4000  # number of cells
r_ei     = 4.0   # number of excitatory cells:number of inhibitory cells
pconn    = 0.01  # connection probability
stim_dur = 50.   # (ms) duration of random stimulation
rate     = 10.  # (Hz) frequency of the random stimulation

dt       = 0.1   # (ms) simulation timestep
tstop    = 1000  # (ms) simulaton duration
delay    = 0.2
n_record = 40
# Cell parameters
area     = 20000. # (µm²)
tau_m    = 20.    # (ms)
cm       = 1.     # (µF/cm²)
g_leak   = 5e-5   # (S/cm²)
E_leak   = -60.  # (mV)
v_thresh = -50.   # (mV)
v_reset  = -60.   # (mV)
t_refrac = 5.     # (ms) (clamped at v_reset)
v_mean   = -60.   # (mV) 'mean' membrane potential, for calculating CUBA weights
tau_exc  = 5.     # (ms)
tau_inh  = 10.    # (ms)

# Synapse parameters
Gexc = 4.     # (nS)
Ginh = 51.    # (nS)
Erev_exc = 0.     # (mV)
Erev_inh = -80.   # (mV)

### what is the synaptic delay???

# === Calculate derived parameters =============================================

area  = area*1e-8                     # convert to cm²
cm    = cm*area*1000                  # convert to nF
Rm    = 1e-6/(g_leak*area)            # membrane resistance in MΩ
assert tau_m == cm*Rm                 # just to check
n_exc = int(round((n*r_ei/(1+r_ei)))) # number of excitatory cells
n_inh = n - n_exc                     # number of inhibitory cells
celltype = sim.IF_cond_exp
w_exc    = Gexc*1e-3              # We convert conductances to uS
w_inh    = Ginh*1e-3

# === Build the network ========================================================
extra = {'threads' : threads,
         'filename': "va.xml",
         'label': 'VA'}

cell_params = {
    'tau_m'      : tau_m,    'tau_syn_E'  : tau_exc,  'tau_syn_I'  : tau_inh,
    'v_rest'     : E_leak,   'v_reset'    : v_reset,  'v_thresh'   : v_thresh,
    'cm'         : cm,       'tau_refrac' : t_refrac}

cell_params['e_rev_E'] = Erev_exc
cell_params['e_rev_I'] = Erev_inh



CSAConnector: libneurosim support not available in NEST.
Falling back on PyNN's default CSAConnector.
Please re-compile NEST using --with-libneurosim=PATH


/home/laurent/.local/lib/python3.8/site-packages/pyNN/nest/__init__.py:53: UserWarning:Unable to install NEST extensions. Certain models may not be available.


## running it...

In [3]:
node_id = sim.setup(timestep=dt, min_delay=delay, max_delay=1.0, **extra)
num_process = sim.num_processes()

timer.start()

print("%s Creating cell populations..." % node_id)

# create a single population of neurons, and then use population views to define
# excitatory and inhibitory sub-populations
all_cells = sim.Population(n_exc + n_inh, celltype(**cell_params), label="All Cells")
exc_cells = all_cells[:n_exc]
exc_cells.label = "Excitatory cells"
inh_cells = all_cells[n_exc:]
inh_cells.label = "Inhibitory cells"

ext_stim = sim.Population(20, sim.SpikeSourcePoisson(rate=rate, duration=stim_dur), label="expoisson")
rconn = 0.01
ext_conn = sim.FixedProbabilityConnector(rconn)
ext_syn = sim.StaticSynapse(weight=0.1)

print("%s Initialising membrane potential to random values..." % node_id)
rng = NumpyRNG(seed=rngseed, parallel_safe=parallel_safe)
uniformDistr = RandomDistribution('uniform', low=v_reset, high=v_thresh, rng=rng)
all_cells.initialize(v=uniformDistr)

print("%s Connecting populations..." % node_id)
progress_bar = ProgressBar(width=20)
connector = sim.FixedProbabilityConnector(pconn, rng=rng, callback=progress_bar)
exc_syn = sim.StaticSynapse(weight=w_exc, delay=delay)
inh_syn = sim.StaticSynapse(weight=w_inh, delay=delay)

connections = {}
connections['exc'] = sim.Projection(exc_cells, all_cells, connector, exc_syn, receptor_type='excitatory')
connections['inh'] = sim.Projection(inh_cells, all_cells, connector, inh_syn, receptor_type='inhibitory')
connections['ext'] = sim.Projection(ext_stim, all_cells, ext_conn, ext_syn, receptor_type='excitatory')

# === Setup recording ==========================================================
print("%s Setting up recording..." % node_id)
all_cells.record('spikes')
exc_cells[[0, 1]].record('v')

buildCPUTime = timer.diff()

# === Run simulation ===========================================================
print("%d Running simulation..." % node_id)
sim.run(tstop)

E_count = exc_cells.mean_spike_count()
I_count = inh_cells.mean_spike_count()

n_connections = "%d e→e,i  %d i→e,i" % (connections['exc'].size(),
                                      connections['inh'].size())
simCPUTime = timer.diff()
# === Output results to file ====================================================

print("%d Writing data to file..." % node_id)

filename = normalized_filename("Results", "VAbenchmarks_exc", "pkl", 'nest', 1)
exc_cells.write_data(filename)
inh_cells.write_data(filename.replace("exc", "inh"))

writeCPUTime = timer.diff()

print("\n--- Vogels-Abbott Network Simulation ---")
print("Nodes                  : %d" % 1)
print("Number of Neurons      : %d" % n)
print("Number of Synapses     : %s" % n_connections)
print("Excitatory conductance : %g nS" % Gexc)
print("Inhibitory conductance : %g nS" % Ginh)
print("Excitatory rate        : %g Hz" % (E_count * 1000.0 / tstop,))
print("Inhibitory rate        : %g Hz" % (I_count * 1000.0 / tstop,))
print("Build time             : %g s" % buildCPUTime)
print("Simulation time        : %g s" % simCPUTime)
print("Writing time           : %g s" % writeCPUTime)


# === Finished with simulator ==================================================

sim.end()

2020-12-02 13:15:25,818 WARNING  [PyNN] NEST does not allow setting an initial value for g_ex (/home/laurent/.local/lib/python3.8/site-packages/pyNN/nest/populations.py[174]:_set_initial_value_array)
2020-12-02 13:15:25,825 WARNING  [PyNN] NEST does not allow setting an initial value for g_in (/home/laurent/.local/lib/python3.8/site-packages/pyNN/nest/populations.py[174]:_set_initial_value_array)


0 Creating cell populations...
0 Initialising membrane potential to random values...
0 Connecting populations...
0 Setting up recording...0%                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [4]:
# TODO add watermark